In [59]:
# !pip install folium

In [60]:
### 기본 라이브러리 불러오기
import pandas as pd
import folium

In [61]:
'''
[Step 1] 데이터 준비/ 기본 설정
'''

# 서울시내 중학교 진학률 데이터셋 (출처: 교육???)
file_path = './2016_middle_shcool_graduates_report.xlsx'
df = pd.read_excel(file_path, engine='openpyxl', header=0)

In [62]:
# IPython Console 디스플레이 옵션 설정하기
pd.set_option('display.width', None)  # 출력화면의 너비
pd.set_option('display.max_rows', 100)  # 출력할 행의 개수 한도
pd.set_option('display.max_columns', 10)  # 출력할 열의 개수 한도
pd.set_option('display.max_colwidth', 20)  # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True)  # 유니코드 사용 너비 조정

In [63]:
# 열 이름 배열을 출력
print(df.columns.values)
print('\n')

['Unnamed: 0' '지역' '학교명' '코드' '유형' '주야' '남학생수' '여학생수' '일반고' '특성화고' '과학고'
 '외고_국제고' '예고_체고' '마이스터고' '자사고' '자공고' '기타진학' '취업' '미상' '위도' '경도']




In [64]:
'''
[Step 2] 데이터 탐색
'''

# 데이터 살펴보기
print(df.head())
print('\n')

# 데이터 자료형 확인
print(df.info())
print('\n')

# 데이터 통계 요약정보 확인
print(df.describe())
print('\n')

   Unnamed: 0    지역                               학교명  코드  유형  ...  \
0           0  성북구  서울대학교사범대학부설중학교.....       3  국립  ...   
1           1  종로구  서울대학교사범대학부설여자중학교...     3  국립  ...   
2           2  강남구           개원중학교                     3  공립  ...   
3           3  강남구           개포중학교                     3  공립  ...   
4           4  서초구           경원중학교                     3  공립  ...   

  기타진학  취업   미상       위도        경도  
0    0.004     0  0.000  37.594942  127.038909  
1    0.031     0  0.000  37.577473  127.003857  
2    0.009     0  0.003  37.491637  127.071744  
3    0.019     0  0.000  37.480439  127.062201  
4    0.010     0  0.000  37.510750  127.008900  

[5 rows x 21 columns]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  415 non-null    int64  
 1   지역          415 non-null    object 
 2   학교명         415 non-nul

In [65]:
# 지도에 위치 표시
mschool_map = folium.Map(location=[37.55, 126.98], tiles='Stamen Terrain',
                         zoom_start=12)

In [66]:
# 중학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
    folium.CircleMarker([lat, lng],
                        radius=5,  # 원의 반지름
                        color='brown',  # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',  # 원을 채우는 색
                        fill_opacity=0.7,  # 투명도
                        popup=name
                        ).add_to(mschool_map)

In [67]:
# 지도를 html 파일로 저장하기
mschool_map.save('./seoul_mschool_location.html')

In [68]:
'''
[Step 3] 데이터 전처리
'''

# 원핫인코딩(더미 변수)
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()  # label encoder 생성
onehot_encoder = preprocessing.OneHotEncoder()  # one hot encoder 생성

onehot_location = label_encoder.fit_transform(df['지역'])
onehot_code = label_encoder.fit_transform(df['코드'])
onehot_type = label_encoder.fit_transform(df['유형'])
onehot_day = label_encoder.fit_transform(df['주야'])

df['location'] = onehot_location
df['code'] = onehot_code
df['type'] = onehot_type
df['day'] = onehot_day

df.head()

,Unnamed: 0,지역,학교명,코드,유형,...,경도,location,code,type,day
0,0,성북구,서울대학교사범대학부설중학교...,3,국립,...,127.038909,16,0,1,0
1,1,종로구,서울대학교사범대학부설여자중학교...,3,국립,...,127.003857,22,0,1,0
2,2,강남구,개원중학교,3,공립,...,127.071744,0,0,0,0
3,3,강남구,개포중학교,3,공립,...,127.062201,0,0,0,0
4,4,서초구,경원중학교,3,공립,...,127.008900,14,0,0,0


In [69]:
'''
[Step 4] DBSCAN 군집 모형 - sklearn 사용
'''

# sklearn 라이브러리에서 cluster 군집 모형 가져오기
from sklearn import cluster

# 분석에 사용할 속성을 선택 (과학고, 외고국제고, 자사고 진학률)
columns_list = [10, 11, 12]
X = df.iloc[:, columns_list]
print(X[:5])
print('\n')

   과학고  외고_국제고  예고_체고
0   0.018        0.007      0.000
1   0.000        0.035      0.008
2   0.009        0.012      0.003
3   0.013        0.013      0.019
4   0.007        0.010      0.005




In [70]:
# 설명 변수 데이터를 정규화
X = preprocessing.StandardScaler().fit(X).transform(X)
print(X[:5])
print('\n')

[[ 2.02375287 -0.57972902 -0.18926749]
 [-0.65047921  1.84782097 -0.10221179]
 [ 0.68663683 -0.14623795 -0.1566216 ]
 [ 1.28091062 -0.05953974  0.01748981]
 [ 0.38949993 -0.31963438 -0.13485767]]




In [71]:
# DBSCAN 모형 객체 생성
dbm = cluster.DBSCAN(eps=0.2, min_samples=5)

# 모형 학습
dbm.fit(X)

# 예측 (군집)
cluster_label = dbm.labels_
cluster_label

array([-1,  0,  1, -1,  1,  2, -1, -1, -1, -1, -1, -1, -1,  0,  1,  5, -1,
        1, -1, -1,  1,  1,  0, -1, -1,  5, -1, -1, -1,  0, -1, -1,  1,  1,
        0,  0,  1,  1, -1, -1, -1, -1, -1,  0,  1, -1,  3,  0,  1,  5, -1,
        0,  2,  2,  4, -1,  0, -1,  0,  0,  1,  0,  0,  0, -1,  1,  0,  0,
        1, -1,  0, -1, -1,  0,  0,  1,  1, -1,  0,  1,  0,  0,  0,  0,  0,
       -1,  5, -1,  0, -1,  0, -1, -1,  0,  1,  0, -1,  0,  0, -1, -1, -1,
       -1,  3,  0,  1,  0,  0,  1, -1, -1,  0, -1, -1, -1, -1,  1,  0,  3,
        1,  1,  0,  1,  0,  0, -1,  3,  1, -1,  0,  0,  0,  0,  1,  1,  0,
        0, -1,  1,  0,  0, -1,  1, -1,  1,  0,  2,  1,  4,  0,  0,  6,  4,
        0, -1,  0,  0, -1,  0,  0,  3,  1,  0,  4,  0,  0,  4,  0,  1,  0,
        0,  0, -1,  0, -1,  3,  0,  0,  3, -1,  0,  0,  1,  0,  1, -1,  0,
        0,  1, -1,  2,  0,  0,  0,  6,  0,  5,  0, -1, -1,  0,  0,  1,  0,
        0,  0,  1,  1,  0,  6,  0,  4,  5,  0,  1,  0,  0, -1, -1,  1, -1,
       -1, -1,  1,  1, -1

In [72]:
# 예측 결과를 데이터프레임에 추가
df['Cluster'] = cluster_label
df

,Unnamed: 0,지역,학교명,코드,유형,...,location,code,type,day,Cluster
0,0,성북구,서울대학교사범대학부설중학교...,3,국립,...,16,0,1,0,-1
1,1,종로구,서울대학교사범대학부설여자중학교...,3,국립,...,22,0,1,0,0
2,2,강남구,개원중학교,3,공립,...,0,0,0,0,1
3,3,강남구,개포중학교,3,공립,...,0,0,0,0,-1
4,4,서초구,경원중학교,3,공립,...,14,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
410,410,강남구,국립국악중학교,9,국립,...,0,2,1,0,-1
411,411,금천구,국립전통예술중학교,9,국립,...,7,2,1,0,-1
412,412,광진구,선화예술학교,9,사립,...,5,2,2,0,-1
413,413,중구,예원학교,9,사립,...,23,2,2,0,-1


In [73]:
#클러스터 값으로 그룹화하고, 그룹별로 내용 출력 (첫 5행만 출력)
grouped_cols = [0, 1, 3] + columns_list
grouped = df.groupby('Cluster')
for key, group in grouped:
    print('* key :', key)
    print('* number :', len(group))
    print(group.iloc[:, grouped_cols].head())
    print('\n')

* key : -1
* number : 109
   Unnamed: 0    지역  코드  과학고  외고_국제고  예고_체고
0           0  성북구     3   0.018        0.007      0.000
3           3  강남구     3   0.013        0.013      0.019
6           6  강남구     3   0.015        0.036      0.005
7           7  강남구     3   0.032        0.005      0.000
8           8  강남구     3   0.013        0.029      0.009


* key : 0
* number : 157
    Unnamed: 0    지역  코드  과학고  외고_국제고  예고_체고
1            1  종로구     3     0.0        0.035      0.008
13          13  서초구     3     0.0        0.022      0.000
22          22  강남구     3     0.0        0.019      0.000
29          29  강남구     3     0.0        0.007      0.007
34          34  강남구     3     0.0        0.016      0.011


* key : 1
* number : 75
    Unnamed: 0    지역  코드  과학고  외고_국제고  예고_체고
2            2  강남구     3   0.009        0.012      0.003
4            4  서초구     3   0.007        0.010      0.005
14          14  서초구     3   0.010        0.013      0.003
17          17  서초구     3   0.003     

In [58]:
# X2 데이터셋에 대하여 위의 과정을 반복(과학고, 외고국제고, 자사고 진학률 + 유형)
columns_list2 = [10, 11, 14, 23]
X2 = df.iloc[:, columns_list2]
X2[:5]

,과학고,외고_국제고,자사고,type
0,0.018,0.007,0.227,1
1,0.000,0.035,0.043,1
2,0.009,0.012,0.090,0
3,0.013,0.013,0.065,0
4,0.007,0.010,0.282,0


In [27]:


X2 = preprocessing.StandardScaler().fit(X2).transform(X2)
dbm2 = cluster.DBSCAN(eps=0.2, min_samples=5)
dbm2.fit(X2)
df['Cluster2'] = dbm2.labels_

grouped2_cols = [0, 1, 3] + columns_list2
grouped2 = df.groupby('Cluster2')
for key, group in grouped2:
    print('* key :', key)
    print('* number :', len(group))
    print(group.iloc[:, grouped2_cols].head())
    print('\n')

# 분석에 사용할 속성을 선택 (과학고, 외고국제고)
columns_list3 = [10, 11]
X3 = df.iloc[:, columns_list3]
print(X3[:5])
print('\n')

X3 = preprocessing.StandardScaler().fit(X3).transform(X3)
dbm3 = cluster.DBSCAN(eps=0.2, min_samples=5)
dbm3.fit(X3)
df['Cluster3'] = dbm3.labels_

grouped3_cols = [0, 1, 3] + columns_list3
grouped3 = df.groupby('Cluster3')
for key, group in grouped3:
    print('* key :', key)
    print('* number :', len(group))
    print(group.iloc[:, grouped3_cols].head())
    print('\n')

[-1  0  1 -1  1  2 -1 -1 -1 -1 -1 -1 -1  0  1  5 -1  1 -1 -1  1  1  0 -1
 -1  5 -1 -1 -1  0 -1 -1  1  1  0  0  1  1 -1 -1 -1 -1 -1  0  1 -1  3  0
  1  5 -1  0  2  2  4 -1  0 -1  0  0  1  0  0  0 -1  1  0  0  1 -1  0 -1
 -1  0  0  1  1 -1  0  1  0  0  0  0  0 -1  5 -1  0 -1  0 -1 -1  0  1  0
 -1  0  0 -1 -1 -1 -1  3  0  1  0  0  1 -1 -1  0 -1 -1 -1 -1  1  0  3  1
  1  0  1  0  0 -1  3  1 -1  0  0  0  0  1  1  0  0 -1  1  0  0 -1  1 -1
  1  0  2  1  4  0  0  6  4  0 -1  0  0 -1  0  0  3  1  0  4  0  0  4  0
  1  0  0  0 -1  0 -1  3  0  0  3 -1  0  0  1  0  1 -1  0  0  1 -1  2  0
  0  0  6  0  5  0 -1 -1  0  0  1  0  0  0  1  1  0  6  0  4  5  0  1  0
  0 -1 -1  1 -1 -1 -1  1  1 -1  0  0 -1  1  0  1 -1 -1  0 -1 -1 -1 -1  0
 -1  6 -1  1  1  1  0  4  0  0  0  0  1  0  0  6  1  0  1  0  0  1 -1  3
 -1 -1  0  0  1  4  0  0  0  0  4 -1  1  0  1  3  4  0  3  0  0  0  3 -1
  1  1  0  1  6 -1  1  0 -1  0  3  0  0  0  1  0  0  0  0  0  1  0  1  0
  0  0  0  0  0  0  3  0 -1  0 -1  6  0  1  1  0  0